<a href="https://colab.research.google.com/github/shailavij/NLP/blob/master/Stackoverflow_MultiLabel_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/stackoverflow/

Stackoverflow_MultiLabel_TEXT_Classification


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier


In [ ]:
df =pd.read_csv("https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/stackoverflow.csv",index_col=0)

In [ ]:
df.head()

,Text,Tags
2,aspnet site maps has anyone got experience cre...,"['sql', 'asp.net']"
4,adding scripting functionality to net applicat...,"['c#', '.net']"
5,should i use nested classes in this case i am ...,['c++']
6,homegrown consumption of web services i have b...,['.net']
8,automatically update version number i would li...,['c#']


In [ ]:
# Datatype of columns
df.dtypes

Text    object
Tags    object
dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48976 entries, 2 to 1263454
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    48976 non-null  object
 1   Tags    48976 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [ ]:
df.shape

(48976, 2)

In [ ]:
df['Tags'].iloc[0]

"['sql', 'asp.net']"

In [ ]:
# Literal Evaluation : To convert string to List 
import ast

In [ ]:
ast.literal_eval(df['Tags'].iloc[0])

['sql', 'asp.net']

In [ ]:
# TO update entire dataframe 'Tag column' into list , use Apply , lambda fn.
df['Tags'] = df ['Tags'].apply(lambda x :ast.literal_eval(x))

In [ ]:
df.head()

,Text,Tags
2,aspnet site maps has anyone got experience cre...,"[sql, asp.net]"
4,adding scripting functionality to net applicat...,"[c#, .net]"
5,should i use nested classes in this case i am ...,[c++]
6,homegrown consumption of web services i have b...,[.net]
8,automatically update version number i would li...,[c#]


In [ ]:
y=df['Tags']
y

2          [sql, asp.net]
4              [c#, .net]
5                   [c++]
6                  [.net]
8                    [c#]
                ...      
1262668             [c++]
1262834             [c++]
1262915          [python]
1263065          [python]
1263454             [c++]
Name: Tags, Length: 48976, dtype: object

In [ ]:
mulitlable= MultiLabelBinarizer()


In [ ]:
y=mulitlable.fit_transform(df['Tags'])

In [ ]:
y

array([[0, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
mulitlable.classes_

array(['.net', 'android', 'asp.net', 'c', 'c#', 'c++', 'css', 'html',
       'ios', 'iphone', 'java', 'javascript', 'jquery', 'mysql',
       'objective-c', 'php', 'python', 'ruby', 'ruby-on-rails', 'sql'],
      dtype=object)

In [ ]:
pd.DataFrame(y, columns=mulitlable.classes_)

,.net,android,asp.net,c,c#,c++,css,html,ios,iphone,java,javascript,jquery,mysql,objective-c,php,python,ruby,ruby-on-rails,sql
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48971,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48972,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48973,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
48974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
# TF_IDF Vectorizer
tfidf = TfidfVectorizer(analyzer='word',max_features=5000,ngram_range=(1,2))
X=tfidf.fit_transform(df['Text'])


In [ ]:
X

<48976x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 4147809 stored elements in Compressed Sparse Row format>

In [ ]:
# Vocabulary list all words tokenised by tfidf
#tfidf.vocabulary_

In [ ]:
X.shape , y .shape

((48976, 5000), (48976, 20))

In [ ]:
X_train,X_test, Y_train,Y_test= train_test_split(X,y,test_size=0.2, random_state=0)

In [ ]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((39180, 5000), (9796, 5000), (39180, 20), (9796, 20))

# Build Model

In [ ]:
sgd= SGDClassifier()
lr=LogisticRegression(solver='lbfgs')
svc= LinearSVC()

In [ ]:
# Jaccard Similarity to measure performance of multilable
def j_score(y_true, y_pred):
  jaccard = np.minimum(y_true,y_pred).sum(axis=1)/np.maximum(y_true,y_pred).sum(axis=1)
  return jaccard.mean()*100

def print_score(y_pred , clf):
  print("Clf: ",clf.__class__.__name__)
  print('jaccard score: {}'. format(j_score(Y_test,y_pred)))
  print('---')

In [ ]:
for classifier in [sgd,lr,svc]:
  clf = OneVsRestClassifier(classifier)
  clf.fit(X_train,Y_train)
  y_pred= clf.predict(X_test)
  print_score(y_pred,classifier)



Clf:  SGDClassifier
jaccard score: 48.33945828229209
---


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Clf:  LogisticRegression
jaccard score: 48.567782768476924
---
Clf:  LinearSVC
jaccard score: 58.14328977814074
---


Model Test with Real Data!

In [ ]:
x= [ 'how to write ml code in python and java i have data  but do not know how to do it']

In [ ]:
xt=tfidf.transform(x)


In [ ]:
pred1= clf.predict(xt)
pred1

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

In [ ]:
mulitlable.inverse_transform(clf.predict(xt))

[('java', 'python')]